# XGBoost Hyperparameters

In [1]:
import pandas as pd
df = pd.read_csv('heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
# Build a classifier to obtain a baseline score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Initialize the classifier with booster='gbtree' and objective='binary:logistic'
model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2)

# Score the baseline model using cross_val_scores and numpy
from sklearn.model_selection import cross_val_score
import numpy as np
scores = cross_val_score(model, X, y, cv=5)
print('Accuracy:', np.round(scores, 2))
print('Mean Accuracy:', np.round(np.mean(scores), 2))

Accuracy: [0.82 0.8  0.77 0.78 0.77]
Mean Accuracy: 0.79


In [5]:
# Using StratifiedKFold helps ensure that the folds are representative of the whole dataset
# This ensures consistent results across cross_val_score, GridSearchCV, and RandomizedSearchCV.
# Import and initialize the StratifiedKFold object
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [6]:
# Score the baseline model with consistent folds.
scores = cross_val_score(model, X, y, cv=kfold)
print('Accuracy:', np.round(scores, 2))
print('Mean Accuracy:', np.round(np.mean(scores), 2))

Accuracy: [0.72 0.77 0.75 0.85 0.87]
Mean Accuracy: 0.79


In [8]:
# Combining GridSearchCV with RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Define a grid_search function with the params dictionary as input, along with random=False
def grid_search(params, random=False):
    xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, n_jobs=-1)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    grid.fit(X, y)
    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Best score:", np.round(best_score, 3))
